In [1]:
# Define the project directory inorder to complie with the reproduceability of the code
project_path = """/home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/"""

In [2]:
# Import all the required libraries
import warnings
warnings.filterwarnings('ignore')

import gc
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

2023-03-31 15:26:29.472543: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 15:26:29.501789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 15:26:29.918887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv(project_path + "Daphnet-Freezing-of-Gait-Dataset/consolidated_dataset.csv", sep='\t', header=0)
df

,Time in millisecond,Ankle_acc - horizontal forward,Ankle_acc - vertical,Ankle_acc - horizontal lateral,Upper_leg_acc - horizontal forward,Upper_leg_acc - vertical,Upper_leg_acc - horizontal lateral,Trunk_acc - horizontal forward,Trunk_acc - vertical,Trunk_acc - horizontal lateral,Annotation,File S,File R,File
0,15,70,39,-970,0,0,0,0,0,0,0,1.0,1.0,S01R01
1,31,70,39,-970,0,0,0,0,0,0,0,1.0,1.0,S01R01
2,46,60,49,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
3,62,60,49,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
4,78,50,39,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917882,3020296,-131,107,-960,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917883,3020312,-121,127,-970,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917884,3020328,-141,117,-960,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917885,3020343,-131,127,-980,0,0,0,0,0,0,0,10.0,1.0,S10R01


In [4]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return train_test_split(np.array(X), np.array(y), test_size=0.2, shuffle=False)
# X, Y = split_sequences(main_df['S01R01'], n_steps)

In [5]:
results = dict()
verbose = 1
epochs = 2
batch_size = 128
n_steps = 3
n_features = 10
n_outputs = 10
if "multi-output-stacked-LSTM" not in os.listdir(project_path + "models/"):
    os.mkdir(project_path + "models/multi-output-stacked-LSTM")
if f"n_steps_{n_steps}" not in os.listdir(project_path + "models/multi-output-stacked-LSTM"):
    os.mkdir(project_path + f"models/multi-output-stacked-LSTM/n_steps_{n_steps}")

for file_name in df['File'].unique():
    X_train, X_test, y_train, y_test = split_sequences(df[df['File']==file_name].drop(['Time in millisecond', 
                                                           'File S', 
                                                           'File R', 
                                                           'File'], 
                                                          axis=1).values, 
                           n_steps)
    
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    print(file_name)
    hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    print("Accuracy _------------------------------------")
    temp_results = dict()
    temp_results['pred_loss'] , temp_results['pred_accuracy'] = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    model.save(project_path + f"models/multi-output-stacked-LSTM/n_steps_{n_steps}/{file_name}.tf")
    for i in range(len(hist.history['loss'])):
        temp_results[f"loss_epoch_{i}"] = hist.history['loss'][i]
    for i in range(len(hist.history['accuracy'])):
        temp_results[f"accuracy_epoch_{i}"] = hist.history['accuracy'][i]    
    results[file_name] = temp_results
    del X_train
    del y_train
    del X_test
    del y_test
    del model
    del temp_results
    del hist
    gc.collect()
file_name = "S00R00"
X_train, X_test, y_train, y_test = split_sequences(df.drop(['Time in millisecond', 
                                                            'File S', 
                                                            'File R', 
                                                            'File'], 
                                                            axis=1).values, 
                                    n_steps)

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
print(file_name)
hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
print("Accuracy _------------------------------------")
temp_results = dict()
temp_results['pred_loss'] , temp_results['pred_accuracy'] = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
model.save(project_path + f"models/multi-output-stacked-LSTM/n_steps_{n_steps}/{file_name}.tf")
for i in range(len(hist.history['loss'])):
    temp_results[f"loss_epoch_{i}"] = hist.history['loss'][i]
for i in range(len(hist.history['accuracy'])):
    temp_results[f"accuracy_epoch_{i}"] = hist.history['accuracy'][i]    
results[file_name] = temp_results
del X_train
del y_train
del X_test
del y_test
del model
del temp_results
del hist
gc.collect()


2023-03-31 15:26:31.689735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 15:26:31.704847: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


S01R01
Epoch 1/2
950/950 [==============================] - 7s 6ms/step - loss: 366622.3125 - accuracy: 0.6180
Epoch 2/2
950/950 [==============================] - 6s 6ms/step - loss: 366615.8438 - accuracy: 0.6727
Accuracy _------------------------------------
238/238 [==============================] - 1s 3ms/step - loss: 339957.8750 - accuracy: 0.4722


2023-03-31 15:26:45.772398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,10]
	 [[{{node inputs}}]]
2023-03-31 15:26:45.778055: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,10]
	 [[{{node inputs}}]]
2023-03-31 15:26:45.860124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,100]
	 [[{{node inputs}}]]
2023-03-31 1

INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/multi-output-stacked-LSTM/n_steps_3/S01R01.tf/assets


INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/multi-output-stacked-LSTM/n_steps_3/S01R01.tf/assets


...

S00R00
Epoch 1/2
11987/11987 [==============================] - 76s 6ms/step - loss: 361997.5625 - accuracy: 0.6262
Epoch 2/2
11987/11987 [==============================] - 74s 6ms/step - loss: 361995.2812 - accuracy: 0.6516
Accuracy _------------------------------------
2997/2997 [==============================] - 9s 3ms/step - loss: 363571.9688 - accuracy: 0.5288


2023-03-31 15:32:46.240649: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,10]
	 [[{{node inputs}}]]
2023-03-31 15:32:46.249106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,10]
	 [[{{node inputs}}]]
2023-03-31 15:32:46.350683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,100]
	 [[{{node inputs}}]]
2023-03-31 1

INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/multi-output-stacked-LSTM/n_steps_3/S00R00.tf/assets


INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/multi-output-stacked-LSTM/n_steps_3/S00R00.tf/assets


57275

In [6]:
pd.DataFrame(results).T

,pred_loss,pred_accuracy,loss_epoch_0,loss_epoch_1,accuracy_epoch_0,accuracy_epoch_1
S01R01,339957.87500,0.472185,366622.31250,366615.84375,0.617994,0.672712
S01R02,347427.87500,0.378539,388328.78125,388314.53125,0.196362,0.498044
S02R01,417102.28125,0.306918,375862.59375,375855.46875,0.298711,0.378820
S02R02,392572.37500,0.638072,415014.65625,415000.00000,0.386931,0.440143
S03R01,358614.93750,0.556765,347268.46875,347260.40625,0.662997,0.740310
S03R02,313712.50000,0.012766,369680.68750,369668.03125,0.540656,0.625774
S03R03,263345.93750,0.427423,284065.09375,284057.25000,0.728919,0.805166
S04R01,377344.50000,0.707640,319153.93750,319149.40625,0.747329,0.838709
S05R01,430726.53125,0.613460,399934.37500,399924.65625,0.500000,0.634030
S05R02,364551.12500,0.491340,341992.62500,341988.87500,0.747760,0.769623


In [7]:
if "multi-output-stacked-LSTM" not in os.listdir(project_path + "results"):
    os.mkdir(project_path + f"results/multi-output-stacked-LSTM") 
pd.DataFrame(results).T.to_csv(project_path + f"results/multi-output-stacked-LSTM/n_steps_{n_steps}.csv", header=True, index=False, sep="\t")